In [ ]:
# pip install contextily

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import pandas as pd

# Charger le fichier GeoJSON des aménagements cyclables
gdf = gpd.read_file('data/amenagements-cyclables.geojson')

# Filtrer les données pour Paris en utilisant la colonne 'arrondissement'
gdf_paris = gdf[gdf['arrondissement'].notnull()]  # Filtrer pour Paris

# Filtrer uniquement les aménagements cyclables pertinents
aménagements_cyclables = [
    'bande cyclable', 
    'piste cyclable', 
    'voie verte', 
    'double-sens cyclable simple', 
    'couloir bus ouvert aux vélos'
]

gdf_paris_cyclable = gdf_paris[gdf_paris['amenagement'].isin(aménagements_cyclables)]

### **VERIF DISTANCES**

In [ ]:
# Reprojeter en EPSG:2154 pour un calcul de longueur précis en mètres
gdf_paris_cyclable = gdf_paris_cyclable.to_crs(epsg=2154)

# Séparer les données selon la valeur de 'voie_a_sens_unique'
gdf_paris_cyclable_non_unique = gdf_paris_cyclable[gdf_paris_cyclable['voie_a_sens_unique'] == 'non']
gdf_paris_cyclable_unique = gdf_paris_cyclable[gdf_paris_cyclable['voie_a_sens_unique'] != 'non']

# Pour les aménagements à double sens, conserver les lignes les plus récentes
gdf_paris_cyclable_non_unique = gdf_paris_cyclable_non_unique.sort_values(by='date_export', ascending=False)
gdf_paris_cyclable_non_unique = gdf_paris_cyclable_non_unique.drop_duplicates(subset=['osm_id', 'cote_amenagement'])

# Pour les aménagements à sens unique, conserver uniquement la ligne la plus récente
gdf_paris_cyclable_unique = gdf_paris_cyclable_unique.sort_values(by='date_export', ascending=False)
gdf_paris_cyclable_unique = gdf_paris_cyclable_unique.drop_duplicates(subset=['osm_id'])

# Combiner les deux DataFrames traités
gdf_paris_cyclable_final = pd.concat([gdf_paris_cyclable_non_unique, gdf_paris_cyclable_unique])

# Calculer la longueur des aménagements cyclables (en kilomètres)
gdf_paris_cyclable_final['longueur'] = gdf_paris_cyclable_final.geometry.length / 1000  # Conversion des mètres en kilomètres

# Grouper par type d'aménagement et sommer les longueurs
longueur_par_type = gdf_paris_cyclable_final.groupby('amenagement')['longueur'].sum()

# Afficher les longueurs par type d'aménagement en kilomètres
print("Longueur par type d'aménagement cyclable à Paris (en km) :")
for amenagement, longueur in longueur_par_type.items():
    print(f"{amenagement}: {longueur:.2f} km")

# Calculer et afficher la longueur totale des aménagements cyclables (en km)
total_length = longueur_par_type.sum()
print(f"\nLongueur totale des aménagements cyclables à Paris : {total_length:.2f} km")

### **LECTURE CARTE VIA GEOJSON**

In [ ]:
gdf_paris_web_mercator.head()

In [ ]:
# Reprojeter en EPSG:3857 pour la visualisation sur la carte
gdf_paris_web_mercator = gdf_paris_cyclable_final.to_crs(epsg=3857)

# Créer une figure et un axe pour la carte
fig, ax = plt.subplots(figsize=(10, 8))

# Tracer les aménagements cyclables
gdf_paris_web_mercator.plot(ax=ax, alpha=0.7, linewidth=1, column='amenagement', legend=True)

# Ajouter le fond de carte
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

# Titre et mise en page
plt.title('Aménagements cyclables à Paris', fontsize=16)
plt.axis('off')

# Ajuster les limites de la carte aux données de Paris
ax.set_xlim(gdf_paris_web_mercator.total_bounds[[0, 2]])
ax.set_ylim(gdf_paris_web_mercator.total_bounds[[1, 3]])

# Afficher la carte
plt.tight_layout()
plt.show()

In [ ]:
gdf_paris_cyclable.head()

In [ ]:
# gdf_paris_cyclable['osm_id'].value_counts().sort_values(ascending=False)

gdf_paris_cyclable[gdf_paris_cyclable['osm_id'] == 41486595]

### **SAVE FILE**

In [ ]:
# Sauvegarder les résultats pour Paris dans un nouveau fichier GeoJSON
gdf_paris_cyclable.to_file('data/paris_amenagements_cyclables.geojson', driver='GeoJSON')

In [ ]:
gdf_paris_cyclable_final.to_csv('paris_pistes_cyclables.csv', index=False, sep=';')